In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../Data/Final_Thawra.csv', header=1, index_col='recordnumber')
df.head(2)

,Initials,datecoded,narrative,nameofevent,Startdate,endate,ongoing,timeofprotest,time_range,governorate,...,pagenum1_restored,URL1_restored,Unnamed: 158,drive_text,Unnamed: 160,drive,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165
recordnumber,,,,,,,,,,,,,,,,,,,,,
20190001.0,HAM,2022-11-23,CGTL and Sabaa Party organize a nation wide st...,NaN,2019-01-04,NaN,0.0,NaN,Unspecified,National,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190002.0,HAM,2023-01-12,اهالي تمنين التحتا يقطعون الطريق على تقاطع الم...,NaN,2019-01-04,NaN,NaN,NaN,Unspecified,Beqaa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# df['drive'] = ''
# for _, row in df.iterrows():
#     for i in row.index:
#         if str(row[i]).startswith('https://drive.google'):
#             df.at[_,'drive'] = row[i]

In [5]:
df.loc[20200924.0][-10:]

pagenum1_restored                                                  NaN
URL1_restored                                                      NaN
Unnamed: 158                                                       NaN
drive_text                                                         NaN
Unnamed: 160                                                       NaN
drive                https://drive.google.com/open?id=11WKaYgWel8yQ...
Unnamed: 162                                                       NaN
Unnamed: 163                                                       NaN
Unnamed: 164                                                       NaN
Unnamed: 165                                                       NaN
Name: 20200924.0, dtype: object

In [ ]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)

In [ ]:
def extract_text_from_pdf(pdf_content):
  resource_manager = PDFResourceManager()
  fake_file_handle = io.StringIO()
  converter = TextConverter(resource_manager, fake_file_handle)
  page_interpreter = PDFPageInterpreter(resource_manager, converter)

  with io.BytesIO(pdf_content) as fh:
    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
      page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

  # close open handles
  converter.close()
  fake_file_handle.close()

  return text